### 문제1: 117쪽 문제 3.13

<표 3.2>의 참게 예제 자료를 www.stat.ufl.edu/~aa/cat/data 웹사이트에서 다운로드하여 x=무게를 예측변수로 간주하고 Y=부수체의 수를 반응변수로 간주하여 포아송 로그 선형모형을 적합하라.

In [5]:
# crab은 연번
# sat는 부수체의 수
# weight는 무게
# width는 등딱지너비
# color는 색깔(1=약간밝은색, 2=중간색, 3=약간어두운색, 4=어두운색)
# spine은 등뼈의 상태(1=둘다양호함, 2=1개가 휘거나 좋지않음, 3=둘다 휘거나 좋지않음)
# y값도 이미 입력되어 있다.

crab_data = read.table("Crabs.dat.txt", header=TRUE)
head(crab_data)

# 포아송 로그 선형모형 적합하기
model1 = glm(sat ~ weight, family=poisson(link="log"), data=crab_data)

summary(model1)

,crab,sat,y,weight,width,color,spine
,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>
1,1,8,1,3.05,28.3,2,3
2,2,0,0,1.55,22.5,3,3
3,3,9,1,2.30,26.0,1,1
4,4,0,0,2.10,24.8,3,3
5,5,4,1,2.60,26.0,3,3
6,6,0,0,2.10,23.8,2,3



Call:
glm(formula = sat ~ weight, family = poisson(link = "log"), data = crab_data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.42841    0.17893  -2.394   0.0167 *  
weight       0.58930    0.06502   9.064   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 632.79  on 172  degrees of freedom
Residual deviance: 560.87  on 171  degrees of freedom
AIC: 920.16

Number of Fisher Scoring iterations: 5


#### a. 예측식을 구하고 암참게의 평균 무게가 2.44 kg일 때 부수체의 평균수를 추정하라.

포아송 로그 선형모형의 예측식은 다음과 같다.

<img src="https://latex.codecogs.com/svg.image?log(\hat{sat})=\alpha+\beta\times weight">

예측식은 다음과 같이 변형할 수 있다.

<img src="https://latex.codecogs.com/svg.image?\hat{sat}=exp(\alpha+\beta\times weight)">

모델 요약 결과를 보면 α = -0.428, β = 0.589 이므로 예측식은 다음과 같다.

<img src="https://latex.codecogs.com/svg.image?\hat{sat}=exp(-0.428+0.589\times weight)">

weight = 2.44 이면 
exp(-0.428+0.589*2.44) = 2.74

weight = 2.44일때 부수체의 평균수 추정값은 2.74개 이다.

In [13]:
# weight=2.44 일때 sat 예측값 구하기
input_data = data.frame(weight=2.44)
result = predict(model1, newdata=input_data)

cat("log(sat):", result, "\n")
cat("sat:", exp(result))

log(sat): 1.009497 
sat: 2.74422

#### c. <img src="https://latex.codecogs.com/svg.image?\hat\beta">를 이용하여 무게의 효과를 설명하라. 모수 <img src="https://latex.codecogs.com/svg.image?\beta">와 무게가 1 kg 증가할 때마다 발생하는 승법 효과에 대한 95% 신뢰구간을 구하라.

< 무게가 1증가할 떄의 효과 >

<img src="https://latex.codecogs.com/svg.image?\hat{\beta}">는 예측변수인 무게가 1 증가할 때 반응변수인 부수체 수가 얼마나 변하는지를 나타냅니다. 여기서는 로그 선형모형이기 때문에 무게가 1 증가할 때 반응변수의 "로그값"이 얼마나 변하는지 나타냅니다.

실제로 무게가 1kg 증가하면,
<img src="https://latex.codecogs.com/svg.image?exp(\hat{\beta})=exp(0.589)\approx1.802">
부수체의 수는 80.2% 증가한다고 볼 수 있습니다.

< 승법 효과에 대한 95% 신뢰구간 >
승법 효과의 신뢰구간은 <img src="https://latex.codecogs.com/svg.image?\hat{\beta}">의 표준오차(0.065)를 이용하여 구할 수 있습니다.

승법 효과의 95% 신뢰구간

[<img src="https://latex.codecogs.com/svg.image?\hat{\beta}-1.96\times SE(\hat{\beta}),\hat{\beta}+1.96\times SE(\hat{\beta})">]

=[<img src="https://latex.codecogs.com/svg.image?0.589-1.96\times 0.065,0.589+1.96\times 0.065">]

=[0.462, 0.717]

로그 선형모형이므로 이 값을 지수 변환하면 [1.587, 2.048] 이다.


In [17]:
# beta 값과 SE(beta) 값 입력
beta = coef(summary(model1))["weight", "Estimate"]
se_beta = coef(summary(model1))["weight", "Std. Error"]

# 신뢰구간 계산
lo_value = beta - 1.96 * se_beta
hi_value = beta + 1.96 * se_beta

# 신뢰구간 표시
cat("lo_value:", lo_value, ",", "hi_value:", hi_value, "\n")
cat("exo(lo_value):", exp(lo_value), ",", "exp(hi_value):", exp(hi_value))

lo_value: 0.4618719 , hi_value: 0.7167364 
exo(lo_value): 1.587042 , exp(hi_value): 2.047739

#### d. 부수체의 평균수와 무게와의 독립성을 왈드 검정과 가능도비 검정을 하고 결과를 해석하라.

In [18]:
summary(model1)


Call:
glm(formula = sat ~ weight, family = poisson(link = "log"), data = crab_data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.42841    0.17893  -2.394   0.0167 *  
weight       0.58930    0.06502   9.064   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 632.79  on 172  degrees of freedom
Residual deviance: 560.87  on 171  degrees of freedom
AIC: 920.16

Number of Fisher Scoring iterations: 5


< 왈드 검정 >
왈드 검정의 통계량은 <img src="https://latex.codecogs.com/svg.image?z=\frac{\hat\beta}{SE(\hat\beta)}"> 이다.

z = 0.589 / 0.0650 = 9.064

이고, z는 귀무가설 하에서 표준정규분포를 따른다.
이 때 p값은 0에 아주 가까운 값(2e-16)이므로 귀무가설을 기각할 수 있다.
즉, 무게는 부수체 수에 유의미한 영향을 미친다.

< 가능도비 검정 >
가능도비 검정은 β=0 일 때의 로그가능도 함수의 최대값(L0)과 β에 대한 제한조건이 없는 경우의 로그가능도 함수의 최대값(L1)을 비교하여 검정을 실시한다.
가능도비 검정의 검정통계량은 <img src="https://latex.codecogs.com/svg.image?-2(L_0 - L_1)"> 이며, 검정통계량은 자유도가 1인 카이제곱분포를 따른다.

이 문제에서 검정통계량은 71.925이고, p값은 0에 아주 가까운 값(2.235e-17)이므로 귀무가설을 기각할 수 있다.
즉, 무게는 부수체 수에 유의미한 영향을 미친다.

In [25]:
# beta=0인 경우의 모델(단순 모델)
model2 = glm(sat~1, family=poisson(link="log"), data=crab_data)

# 가능도 계산
L_1 = logLik(model1)
L_0 = logLik(model2)

# 가능도비 검정 통계량
l_value = -2 * (L_0 - L_1)

# p값 계산
p_value = 1-pchisq(l_value, df=1)

# 결과
cat("l_value:", l_value, "\n")
cat("p_value:", p_value)

# anova 함수를 이용하여 계산하기
lr_test = anova(model2, model1, test="LRT")
lr_test

l_value: 71.92524 
p_value: 0

,Resid. Df,Resid. Dev,Df,Deviance,Pr(>Chi)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,172,632.7917,NA,NA,NA
2,171,560.8664,1,71.92524,2.235066e-17


### 문제2: 151쪽 문제 4.5

1986년의 챌린저호 사고 전에 있었던 23회의 우주선 비행에 대해, <표 4.6>과 이 책의 웹사이트에 있는 Shuttle 자료 파일은 비행시의 기온(˚F)과 열로 인한 문제가 발생했는지의 여부를 나타낸 것이다.

<표 4.6> 연습문제 4.5의 우주선에 대한 자료
Ft, Temp, TD
1,66,0
2,70,1
3,69,0
4,68,0
5,67,0
6,72,0
7,73,0
8,70,0
9,57,1
10,63,1
11,70,1
12,78,0
13,67,0
14,53,1
15,67,0
16,75,0
17,70,0
18,81,0
19,76,0
20,79,0
21,75,1
22,76,0
23,58,1

주의: Ft=항공기 번호, Temp=기온, TD=열 문제 발생 여부(1=예, 0=아니오)

In [28]:
# 데이터 프레임 생성
fly_data <- data.frame(
  Ft = 1:23,
  Temp = c(66, 70, 69, 68, 67, 72, 73, 70, 57, 63, 70, 78, 67, 53, 67, 75, 70, 81, 76, 79, 75, 76, 58),
  TD = c(0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1)
)

# 데이터 확인
summary(fly_data)

       Ft            Temp             TD        
 Min.   : 1.0   Min.   :53.00   Min.   :0.0000  
 1st Qu.: 6.5   1st Qu.:67.00   1st Qu.:0.0000  
 Median :12.0   Median :70.00   Median :0.0000  
 Mean   :12.0   Mean   :69.57   Mean   :0.3043  
 3rd Qu.:17.5   3rd Qu.:75.00   3rd Qu.:1.0000  
 Max.   :23.0   Max.   :81.00   Max.   :1.0000  

### a. 로지스틱 회귀모형을 이용하여 열 문제가 발생할 확률에 대한 기온의 효과를 모형화하고 적합 결과를 해석하라.

In [29]:
# 로지스틱 회귀모형 적합
model3 = glm(TD ~ Temp, data=fly_data, family=binomial(link="logit"))

# 적합 결과
summary(model3)


Call:
glm(formula = TD ~ Temp, family = binomial(link = "logit"), data = fly_data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)  
(Intercept)  15.0429     7.3786   2.039   0.0415 *
Temp         -0.2322     0.1082  -2.145   0.0320 *
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 28.267  on 22  degrees of freedom
Residual deviance: 20.315  on 21  degrees of freedom
AIC: 24.315

Number of Fisher Scoring iterations: 5


< 로지스틱 회귀모형의 적합식 >

<img src="https://latex.codecogs.com/svg.image?logit(P)=log(\frac{P}{1-P})=\alpha+\beta\cdot Temp">

P: 열 문제가 발생(TD=1)할 확률
<img src="https://latex.codecogs.com/svg.image?\alpha=15.043">
<img src="https://latex.codecogs.com/svg.image?\beta=-0.232">

수식을 변형하여 P를 구하면 다음과 같다.
<img src="https://latex.codecogs.com/svg.image?P=\frac{exp(\alpha+\beta\cdot Temp)}{1+exp(\alpha+\beta\cdot Temp)}">
<img src="https://latex.codecogs.com/svg.image?=\frac{exp(15.043-0.232\cdot Temp)}{1+exp(15.043-0.232\cdot Temp)}">

< 기온의 효과 >

기온의 회귀계수는 -0.232이다.  
이는 기온이 1˚F 증가할 때 열 문제가 발생할 로그 오즈가 약 0.232만큼 감소한다는 의미이다.

### b. 챌린저호 비행시의 기온인 31˚F에서 열 문제가 발생할 확률을 예측하라.

Temp = 31을 대입하여 열 문제가 발생할 확률을 직접 계산할 수도 있고,
적합된 모델을 활용하여 로그오즈를 구하고, 확률을 계산할 수도 있다.

In [34]:
# Temp=31을 대입하여 직접 계산
exp(15.043-0.232*31) / (1+exp(15.043-0.232*31))

# 모델을 활용하여 계산하기
input_temp = data.frame(Temp=31)
log_odds = predict(model3, newdata=input_temp, type="link")
prob = exp(log_odds) / (1+exp(log_odds))

prob

[1] 0.9996108

1 
0.9996088

결론적으로 31˚F에서 열 문제가 발생할 확률은 99.961% 이다.

### c. 기온이 몇 도일 때에 예측확률이 0.50이 되는가? 이 기온값에서 기온이 1도 증가함에 따라 나타나는 예측확률의 변화를 나타내는 선형근사식을 구하라.

예측확률 P=0.5일 때 
<img src="https://latex.codecogs.com/svg.image?logit(P)=log(\frac{P}{1-P})=log(\frac{0.5}{1-0.5})=0"> 이 성립한다.

<img src="https://latex.codecogs.com/svg.image?logit(P)=\alpha+\beta\cdot Temp=0"> 역시 성립하므로 예측확률 P=0.5인 Temp는 
<img src="https://latex.codecogs.com/svg.image?Temp=-\frac{\alpha}{\beta}">이다.

In [37]:
alpha = model3$coefficients[1]
beta = model3$coefficients[2]
alpha
beta

Temp_05 = - alpha / beta
Temp_05

(Intercept) 
    15.0429

Temp 
-0.2321627

(Intercept) 
   64.79464

예측 확률이 0.5가 되는 기온은 64.795˚F 이다.

이 기온값에서 기온이 1도 증가함에 따라 나타나는 예측확률의 변화값은 다음과 같이 구할 수 있다.

In [47]:
# 모델을 활용하여 계산하기
input_temp = data.frame(Temp=64.795)
log_odds = predict(model3, newdata=input_temp, type="link")
prob1 = exp(log_odds) / (1+exp(log_odds))

cat("기온이 64.795일 때 확률:", round(prob1,3), "\n")

input_temp = data.frame(Temp=64.795+1)
log_odds = predict(model3, newdata=input_temp, type="link")
prob2 = exp(log_odds) / (1+exp(log_odds))

cat("기온이 65.795일 때 확률:", round(prob2,3), "\n")

cat("예측 확률의 변화값:", round(prob2-prob1, 3))

기온이 64.795일 때 확률: 0.5 


기온이 65.795일 때 확률: 0.442 
예측 확률의 변화값: -0.058

즉 64.795˚F에서 기온이 1˚F 상승하면 열 문제가 발생할 예측확률은 5.8% 감소한다.

예측확률의 선형 근사식은 다음과 같다.

(P-0.5)=-0.058*(Temp-64.795)

### d. 열 문제가 발생할 오즈에 대한 기온의 효과를 해석하라.

열 문제가 발생할 오즈는 다음과 같다.
<img src="https://latex.codecogs.com/svg.image?Odds=\frac{P(TD=1)}{P(TD=0)}=\frac{P}{1-P}">

로지스틱 회귀모델의 식은 다음과 같다.
<img src="https://latex.codecogs.com/svg.image?logit(P)=log(\frac{P}{1-P})=log(Odds)=\alpha+\beta\cdot Temp">

<img src="https://latex.codecogs.com/svg.image?Odds=exp(\alpha+\beta\cdot Temp)">

기온의 효과는 β값으로 설명할 수 있는데, 기온이 1˚F 증가할 때 열 문제가 발생할 오즈는 exp(-0.232) 배 변화한다는 것이다.

exp(-0.232) = 0.793 이므로  
기온이 1˚F 증가하면 열 문제가 발생할 오즈가 20.7% 감소하는 효과가 있다. 


### e. 기온의 효과가 없다는 가설에 대하여 가설검정을 하라.

< 왈드 검정 >
왈드 검정의 통계량은 <img src="https://latex.codecogs.com/svg.image?z=\frac{\hat\beta}{SE(\hat\beta)}"> 이다.

z = -0.232 / 0.108 = -2.145

이고, z는 귀무가설 하에서 표준정규분포를 따른다.
이 때 p값은 0.032이므로 유의수준 0.05일 때 귀무가설을 기각할 수 있다.
즉, 기온은 열 문제 발생에 유의미한 영향을 미친다.

< 가능도비 검정 >
가능도비 검정은 β=0 일 때의 로그가능도 함수의 최대값(L0)과 β에 대한 제한조건이 없는 경우의 로그가능도 함수의 최대값(L1)을 비교하여 검정을 실시한다.
가능도비 검정의 검정통계량은 <img src="https://latex.codecogs.com/svg.image?-2(L_0 - L_1)"> 이며, 검정통계량은 자유도가 1인 카이제곱분포를 따른다.

이 문제에서 검정통계량은 7.952이고, p값은 0.0048이므로 귀무가설을 기각할 수 있다.
즉, 기온은 열 문제 발생에 유의미한 영향을 미친다.

In [51]:
# beta=0인 경우의 모델(단순 모델)
model4 = glm(TD ~ 1, data=fly_data, family=binomial(link="logit"))

# anova 함수를 이용하여 계산하기
lr_test = anova(model4, model3, test="LRT")
lr_test

l_value: 7.95196 
p_value: 0.004803533

,Resid. Df,Resid. Dev,Df,Deviance,Pr(>Chi)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,22,28.26715,NA,NA,NA
2,21,20.31519,1,7.95196,0.004803533
